# Multi-Year Data Correlation Analysis (2020-2024)
## Comprehensive correlation analysis across multiple years

## 0. Setup

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings

warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 8)

print("Libraries loaded successfully")

## 1. Load Merged Data

In [ ]:
# Load merged data (created by merge_and_analyze.py)
df = pd.read_pickle('analysis_results/merged_data_all_years.pkl')

print(f"Data shape: {df.shape[0]:,} rows x {df.shape[1]} columns")
print(f"Memory: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
print(f"\nYear-Quarter range: {df['STDR_YYQU_CD'].min()} - {df['STDR_YYQU_CD'].max()}")
print(f"\nYear-Quarter distribution:")
print(df['STDR_YYQU_CD'].value_counts().sort_index())

df.head()

## 2. Time Zone Correlation Analysis

Analyze how sales across different time zones correlate with multi-year data.

In [ ]:
time_cols = ['TMZON_00_06_SELNG_AMT', 'TMZON_06_11_SELNG_AMT', 'TMZON_11_14_SELNG_AMT',
             'TMZON_14_17_SELNG_AMT', 'TMZON_17_21_SELNG_AMT', 'TMZON_21_24_SELNG_AMT']

time_corr = df[time_cols].corr()

plt.figure(figsize=(10, 8))
sns.heatmap(time_corr, annot=True, fmt='.3f', cmap='RdYlBu_r', center=0,
            square=True, linewidths=1, cbar_kws={"shrink": 0.8},
            xticklabels=['00-06', '06-11', '11-14', '14-17', '17-21', '21-24'],
            yticklabels=['00-06', '06-11', '11-14', '14-17', '17-21', '21-24'])
plt.title('Sales Correlation by Time Zone (Multi-Year 2020-2024)', fontsize=14, fontweight='bold', pad=20)
plt.tight_layout()
plt.show()

print("\nKey Insights:")
print(f"Night hours correlation (17-21 & 21-24): {time_corr.loc['TMZON_17_21_SELNG_AMT', 'TMZON_21_24_SELNG_AMT']:.3f}")
print(f"Late night independence (00-06 avg): {time_corr.loc['TMZON_00_06_SELNG_AMT'].mean():.3f}")

## 3. Day of Week Correlation Analysis

In [ ]:
day_cols = ['MON_SELNG_AMT', 'TUES_SELNG_AMT', 'WED_SELNG_AMT',
            'THUR_SELNG_AMT', 'FRI_SELNG_AMT', 'SAT_SELNG_AMT', 'SUN_SELNG_AMT']

day_corr = df[day_cols].corr()

plt.figure(figsize=(10, 8))
sns.heatmap(day_corr, annot=True, fmt='.3f', cmap='RdYlBu_r', center=0,
            square=True, linewidths=1, cbar_kws={"shrink": 0.8},
            xticklabels=['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun'],
            yticklabels=['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun'])
plt.title('Sales Correlation by Day of Week (Multi-Year 2020-2024)', fontsize=14, fontweight='bold', pad=20)
plt.tight_layout()
plt.show()

# Statistics
weekday_corr = day_corr.loc['MON_SELNG_AMT':'FRI_SELNG_AMT', 'MON_SELNG_AMT':'FRI_SELNG_AMT']
weekday_avg = weekday_corr.values[np.triu_indices_from(weekday_corr.values, k=1)].mean()

print("\nKey Insights:")
print(f"Average weekday correlation: {weekday_avg:.3f}")
print(f"Weekend correlation (Sat-Sun): {day_corr.loc['SAT_SELNG_AMT', 'SUN_SELNG_AMT']:.3f}")

## 4. Age Group Correlation Analysis

In [ ]:
age_cols = ['AGRDE_10_SELNG_AMT', 'AGRDE_20_SELNG_AMT', 'AGRDE_30_SELNG_AMT',
            'AGRDE_40_SELNG_AMT', 'AGRDE_50_SELNG_AMT', 'AGRDE_60_ABOVE_SELNG_AMT']

age_corr = df[age_cols].corr()

plt.figure(figsize=(10, 8))
sns.heatmap(age_corr, annot=True, fmt='.3f', cmap='RdYlBu_r', center=0,
            square=True, linewidths=1, cbar_kws={"shrink": 0.8},
            xticklabels=['10s', '20s', '30s', '40s', '50s', '60+'],
            yticklabels=['10s', '20s', '30s', '40s', '50s', '60+'])
plt.title('Sales Correlation by Age Group (Multi-Year 2020-2024)', fontsize=14, fontweight='bold', pad=20)
plt.tight_layout()
plt.show()

# Statistics
core_corr = age_corr.loc['AGRDE_20_SELNG_AMT':'AGRDE_40_SELNG_AMT', 'AGRDE_20_SELNG_AMT':'AGRDE_40_SELNG_AMT']
core_avg = core_corr.values[np.triu_indices_from(core_corr.values, k=1)].mean()

print("\nKey Insights:")
print(f"Young adult (20s-30s) correlation: {age_corr.loc['AGRDE_20_SELNG_AMT', 'AGRDE_30_SELNG_AMT']:.3f}")
print(f"Core consumer group (20s-40s) avg: {core_avg:.3f}")
print(f"Senior (50s-60+) correlation: {age_corr.loc['AGRDE_50_SELNG_AMT', 'AGRDE_60_ABOVE_SELNG_AMT']:.3f}")

## 5. Temporal Trends Analysis

In [ ]:
# Aggregate by year-quarter
time_series = df.groupby('STDR_YYQU_CD').agg({
    'THSMON_SELNG_AMT': 'sum',
    'THSMON_SELNG_CO': 'sum'
}).reset_index()

time_series['avg_transaction'] = time_series['THSMON_SELNG_AMT'] / time_series['THSMON_SELNG_CO']

fig, axes = plt.subplots(2, 1, figsize=(14, 10))

# Total sales trend
axes[0].plot(time_series['STDR_YYQU_CD'], time_series['THSMON_SELNG_AMT'] / 1e12,
             marker='o', linewidth=2, markersize=10, color='steelblue')
axes[0].set_title('Total Sales Trend (2020-2024)', fontsize=14, fontweight='bold')
axes[0].set_ylabel('Total Sales (Trillion KRW)')
axes[0].grid(True, alpha=0.3)
axes[0].tick_params(axis='x', rotation=45)

# Average transaction trend
axes[1].plot(time_series['STDR_YYQU_CD'], time_series['avg_transaction'] / 1000,
             marker='s', linewidth=2, markersize=10, color='coral')
axes[1].set_title('Average Transaction Amount Trend', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Year-Quarter')
axes[1].set_ylabel('Avg Transaction (Thousand KRW)')
axes[1].grid(True, alpha=0.3)
axes[1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

# Growth statistics
total_growth = (time_series['THSMON_SELNG_AMT'].iloc[-1] / time_series['THSMON_SELNG_AMT'].iloc[0] - 1) * 100
avg_growth = (time_series['avg_transaction'].iloc[-1] / time_series['avg_transaction'].iloc[0] - 1) * 100

print(f"\nTotal sales growth: {total_growth:+.2f}%")
print(f"Average transaction growth: {avg_growth:+.2f}%")

## 6. Seasonality Analysis

In [ ]:
# Extract quarter from STDR_YYQU_CD
df['quarter'] = df['STDR_YYQU_CD'].str[-1].astype(int)

# Analyze by quarter
quarter_analysis = df.groupby('quarter').agg({
    'THSMON_SELNG_AMT': 'mean',
    'WKEND_SELNG_AMT': 'mean',
    'MDWK_SELNG_AMT': 'mean'
})

quarter_analysis['weekend_ratio'] = (quarter_analysis['WKEND_SELNG_AMT'] / 
                                      quarter_analysis['THSMON_SELNG_AMT'] * 100)

fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Average sales by quarter
axes[0].bar(quarter_analysis.index, quarter_analysis['THSMON_SELNG_AMT'] / 1e6)
axes[0].set_title('Average Sales by Quarter', fontsize=12, fontweight='bold')
axes[0].set_xlabel('Quarter')
axes[0].set_ylabel('Average Sales (Million KRW)')
axes[0].set_xticks([1, 2, 3, 4])
axes[0].set_xticklabels(['Q1', 'Q2', 'Q3', 'Q4'])

# Weekend ratio by quarter
axes[1].bar(quarter_analysis.index, quarter_analysis['weekend_ratio'], color='coral')
axes[1].set_title('Weekend Sales Ratio by Quarter', fontsize=12, fontweight='bold')
axes[1].set_xlabel('Quarter')
axes[1].set_ylabel('Weekend Sales Ratio (%)')
axes[1].set_xticks([1, 2, 3, 4])
axes[1].set_xticklabels(['Q1', 'Q2', 'Q3', 'Q4'])

plt.tight_layout()
plt.show()

print("\nQuarter Statistics:")
print(quarter_analysis)

## 7. District Type Comparison (Multi-Year)

In [ ]:
district_stats = df.groupby('TRDAR_SE_CD_NM').agg({
    'THSMON_SELNG_AMT': ['sum', 'mean'],
    'TRDAR_CD': 'nunique',
    'WKEND_SELNG_AMT': 'sum',
    'MDWK_SELNG_AMT': 'sum'
})

district_stats.columns = ['total_sales', 'avg_sales', 'num_districts', 'weekend_sales', 'weekday_sales']
district_stats['weekend_ratio'] = (district_stats['weekend_sales'] / 
                                   (district_stats['weekend_sales'] + district_stats['weekday_sales']) * 100)
district_stats = district_stats.sort_values('total_sales', ascending=False)

print("District Type Statistics (Multi-Year):")
print(district_stats)

# Visualization
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Total sales
axes[0].bar(district_stats.index, district_stats['total_sales'] / 1e12)
axes[0].set_title('Total Sales by District Type', fontsize=12, fontweight='bold')
axes[0].set_ylabel('Total Sales (Trillion KRW)')
axes[0].tick_params(axis='x', rotation=45)

# Average sales
axes[1].bar(district_stats.index, district_stats['avg_sales'] / 1e6, color='green')
axes[1].set_title('Average Sales by District Type', fontsize=12, fontweight='bold')
axes[1].set_ylabel('Average Sales (Million KRW)')
axes[1].tick_params(axis='x', rotation=45)

# Weekend ratio
axes[2].bar(district_stats.index, district_stats['weekend_ratio'], color='coral')
axes[2].set_title('Weekend Ratio by District Type', fontsize=12, fontweight='bold')
axes[2].set_ylabel('Weekend Sales Ratio (%)')
axes[2].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

## 8. Comprehensive Correlation Matrix

In [ ]:
# Select key variables
key_vars = [
    'THSMON_SELNG_AMT',
    'MDWK_SELNG_AMT',
    'WKEND_SELNG_AMT',
    'TMZON_17_21_SELNG_AMT',
    'TMZON_21_24_SELNG_AMT',
    'AGRDE_20_SELNG_AMT',
    'AGRDE_30_SELNG_AMT',
    'AGRDE_40_SELNG_AMT',
    'ML_SELNG_AMT',
    'FML_SELNG_AMT'
]

comprehensive_corr = df[key_vars].corr()

plt.figure(figsize=(12, 10))
sns.heatmap(comprehensive_corr, annot=True, fmt='.2f', cmap='RdYlBu_r', center=0,
            square=True, linewidths=1, cbar_kws={"shrink": 0.8})
plt.title('Comprehensive Correlation Matrix (Multi-Year 2020-2024)', fontsize=14, fontweight='bold', pad=20)
plt.tight_layout()
plt.show()

comprehensive_corr

## 9. Summary and Insights

In [ ]:
print("="*60)
print("MULTI-YEAR ANALYSIS SUMMARY (2020-2024)")
print("="*60)

print("\n[Dataset Overview]")
print(f"Total records: {len(df):,}")
print(f"Year-Quarter range: {df['STDR_YYQU_CD'].min()} to {df['STDR_YYQU_CD'].max()}")
print(f"Number of districts: {df['TRDAR_CD'].nunique():,}")
print(f"Number of industries: {df['SVC_INDUTY_CD_NM'].nunique()}")

print("\n[Key Correlations]")
print(f"Night economy (17-21 & 21-24): {time_corr.loc['TMZON_17_21_SELNG_AMT', 'TMZON_21_24_SELNG_AMT']:.3f}")
print(f"Weekday consistency: {weekday_avg:.3f}")
print(f"Core consumers (20s-40s): {core_avg:.3f}")

print("\n[Growth Trends]")
print(f"Total sales growth: {total_growth:+.2f}%")
print(f"Avg transaction growth: {avg_growth:+.2f}%")

print("\n[Recommendations for Feature Engineering]")
print("1. Night economy indicator: Combine 17-21 & 21-24 time zones (high correlation)")
print("2. Core consumer ratio: Aggregate 20s-40s age groups (consistent pattern)")
print("3. Weekend activity ratio: Strong weekday-weekend distinction")
print("4. Temporal stability: Consider quarter-based seasonality")

print("\n" + "="*60)